## Load protobuf models
To build the protobuf files, I ran:
```
cd src;
protoc --python_out=. --proto_path=protobuf protobuf/sample.proto
```

In [1]:
import sys
from google.protobuf.json_format import MessageToJson
import tempfile

sys.path.append("../src")
sys.path.append("../protobuf")

from ipfs import Ipfs
from sample_pb2 import Example, Type

## Create an example data record

In [2]:
example = Example(
    type = Type.FIZZ,
    content = "hello world!"
)

In [3]:
# Note: if the value of id_field_type is set to 1 (default value) it will not be included in the payload
MessageToJson(example, including_default_value_fields=True)

'{\n  "content": "hello world!",\n  "type": "FIZZ"\n}'

## Serialize the example record to prepare to store it in ipfs

In [4]:
content = example.SerializeToString()
content

b'\x12\x0chello world!'

## Try to deserialize it

In [5]:
reader = Example()
reader.ParseFromString(content)
MessageToJson(reader, including_default_value_fields=True)

'{\n  "content": "hello world!",\n  "type": "FIZZ"\n}'

## Write the serialize message to ipfs

In [6]:
ipfs = Ipfs()
filename = "jupyter_test"
ipfs.add(filename, content)
ipfs.write(filename, content)  # overwrite content

## Read the data back from ipfs

In [7]:
result = ipfs.list_files()
result

['Tjupyter_test',
 'identity',
 'jupyter_test',
 'loan',
 'test_directory_2',
 'var']

In [8]:
result = ipfs.read("jupyter_test")
result

b'\x12\x0chello world!'

In [9]:
reader2 = Example()
reader2.ParseFromString(result)
result = MessageToJson(reader2, including_default_value_fields=True)
result

'{\n  "content": "hello world!",\n  "type": "FIZZ"\n}'

In [10]:
reader2.type

0

In [11]:
reader2.content

'hello world!'